In [ ]:
!pip install gdown
import gdown
url = "https://drive.google.com/uc?=dmfkadfkadsnkfdn/dfdae2322"
output_file = "model.sav"
gdown.download(url, output_file, quiet=False)

In [ ]:
generator = tf.keras.models.load_model('generator')
discriminator = tf.keras.models.load_model('discriminator')

def generate_images_test(model, test_input, plot=False):
    prediction = model(test_input, training=True)
    
    display_list = [test_input[0], prediction[0]]
    title = ['Input Image',  'Predicted Image']

    if plot==True:
        plt.figure(figsize=(15, 15))
        for i in range(2):
            plt.subplot(1, 3, i+1)
            plt.title(title[i])
            # Getting the pixel values in the [0, 1] range to plot.
            plt.imshow(display_list[i] * 0.5 + 0.5)
            plt.axis('off')
        plt.show()
    return prediction[0]

content_dir = "Dataset/Testing_Data/"
mask_dir = "/kaggle/working/mask_files"
gen_dir = "/kaggle/working/gen_files"
sub_dir = "submission"
metadata = pd.read_csv(content_dir + "masked_info.csv")
f = open("Final_csv_file.csv","w")
f.write("filename_box_pixel,Value\n")
vals_dict = {
    'filename_box_pixel' : [],
    'Value' : []
}
print(metadata.shape[0])
model = tf.keras.models.load_model('model')
for idx in range(metadata.shape[0]):
    _, filename, mask1_y, mask1_x, mask2_y, mask2_x = metadata.iloc[idx,:].tolist()
    cords_box1 = [(row, col) for row in range(mask1_y,  mask1_y + 75) for col in range(mask1_x,  mask1_x + 75)]
    cords_box2 = [(row, col) for row in range(mask2_y,  mask2_y + 75) for col in range(mask2_x,  mask2_x + 75)]
    box_cords = {'box1' : cords_box1, 'box2' : cords_box2}
    img_path = 'Dataset/Testing_Data/'+filename  # path to test image
    img = cv2.imread(img_path)  # load image using OpenCV
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # convert to RGB format
    img = cv2.resize(img, (256, 256))  # resize to match model input shape
    img = img / 255.0  # normalize pixel values to range [0, 1]
    img = np.expand_dims(img, axis=0)  # add batch dimension
    inpainted_img = model.predict([img,img])
    test_results = inpainted_img[0]  # remove batch dimension
    for box, all_cords in box_cords.items():
        for px_cords in all_cords:
            b = test_results[px_cords[0]][px_cords[1]][0]
            g = test_results[px_cords[0]][px_cords[1]][1]
            r = test_results[px_cords[0]][px_cords[1]][2]
            px_name_b = filename + "_" + box + "_" + str(px_cords[0]) + "_" + str(px_cords[1]) + "_0"
            px_name_g = filename + "_" + box + "_" + str(px_cords[0]) + "_" + str(px_cords[1]) + "_1"
            px_name_r = filename + "_" + box + "_" + str(px_cords[0]) + "_" + str(px_cords[1]) + "_2"
            f.write(px_name_b+","+str(b)+"\n")
            f.write(px_name_g+","+str(g)+"\n")
            f.write(px_name_r+","+str(r)+"\n")
    if idx % 5 == 0:
        print(idx)
f.close()